<a href="https://colab.research.google.com/github/Aisnya975/problemset_aisyah_033/blob/main/week13_soal18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- CONFIGURATION ---
$CPU_Threshold = 80
$Mem_Threshold_GB = 1
$Disk_Threshold_Percent = 10
$Critical_Services = @("W32Time", "WinDefend", "EventLog")
$LogFile = "C:\Monitor\system_monitor.log"
$EmailAdmin = "admin@company.com"

Function Log-Write($Message) {
    $TimeStamp = Get-Date -Format "yyyy-MM-dd HH:mm:ss"
    Add-Content -Path $LogFile -Value "$TimeStamp - $Message"
}

Function Send-Alert($Subject, $Body) {
    # Send-MailMessage -To $EmailAdmin -Subject $Subject -Body $Body -SmtpServer "smtp.company.com"
    Log-Write "ALERT SENT: $Subject"
}

# --- MAIN LOGIC ---

# 1. Check CPU Usage (Avg over 5 samples/seconds to approximate 5 mins check conceptually)
$CPU = Get-WmiObject Win32_Processor | Measure-Object -Property LoadPercentage -Average
If ($CPU.Average -gt $CPU_Threshold) {
    Send-Alert "High CPU Alert" "CPU Usage is at $($CPU.Average)%"
}

# 2. Check Memory
$OS = Get-WmiObject Win32_OperatingSystem
$FreeMemGB = [math]::round($OS.FreePhysicalMemory / 1MB, 2)
If ($FreeMemGB -lt $Mem_Threshold_GB) {
    Send-Alert "Low Memory Alert" "Available Memory: $FreeMemGB GB"
}

# 3. Check Disk Space
$Disk = Get-WmiObject Win32_LogicalDisk -Filter "DeviceID='C:'"
$FreePercent = ($Disk.FreeSpace / $Disk.Size) * 100
If ($FreePercent -lt $Disk_Threshold_Percent) {
    Send-Alert "Low Disk Space" "Free Space on C: is $([math]::round($FreePercent,2))%"
}

# 4. Check Services
Foreach ($Svc in $Critical_Services) {
    $Status = Get-Service -Name $Svc
    If ($Status.Status -ne 'Running') {
        Send-Alert "Service Stopped" "Critical Service $Svc is NOT running!"
    }
}